In [1]:
# Imports
import os 
import pandas as pd
from django.apps import apps as django_apps
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
from edc_appointment.models import Appointment

In [2]:
def switch_to_cohortc_quart(mv):
    #Maternal Visit
#     mv.save()

    old_q_apps = None
    q_ons = None

    try:
        q_ons = OnScheduleCohortCQuarterly.objects.get(
            subject_identifier=mv.subject_identifier)
    except OnScheduleCohortCQuarterly.DoesNotExist:
        print('whoa!', mv.subject_identifier)
    else:
        old_q_apps = Appointment.objects.filter(subject_identifier=q_ons.subject_identifier,
                                             schedule_name=q_ons.schedule_name,
                                             visit_code_sequence=0)
    
    vs = mv.schedule_name.split('_')
    fschedule_name = '_'.join([vs[0], vs[1].replace('enrol', 'quarterly'), vs[2]])
        
    #check new enrollment onschedule model
    print(">>>>>>>>>>>>>", fschedule_name)
    try:
        ons = OnScheduleCohortBQuarterly.objects.get(
            subject_identifier=mv.subject_identifier,
            schedule_name=fschedule_name)
    except:
        print('hmmmmm!', mv.subject_identifier)
    else:
        print('Modify newly generated quart appointments')
        
        if old_q_apps:
            for appt in Appointment.objects.filter(schedule_name=ons.schedule_name,
                                               subject_identifier=ons.subject_identifier):
                ap = old_q_apps.get(visit_code=appt.visit_code)

                try:
                    v1 = MaternalVisit.objects.get(appointment=ap)
                    v1.appointment = appt
                    v1.save()
                except:
                    pass
                
                appt.user_modified = ap.user_modified
                appt.appt_status = ap.appt_status
                appt.appt_reason = ap.appt_reason
                appt.appt_datetime = ap.appt_datetime

                appt.save()

            ssh = SubjectScheduleHistory.objects.get(subject_identifier=old_q_apps[0].subject_identifier,
                                                           schedule_name=old_q_apps[0].schedule_name)
            ssh.delete()

            old_q_apps.delete()
            q_ons.delete()


In [3]:
def switch_to_cohortc_enroll(subject_identifier, new_cohort):
    #Change cohort in consent
    child_consent = CaregiverChildConsent.objects.filter(subject_identifier=subject_identifier).latest(
        'consent_datetime')
    child_consent.cohort = new_cohort
    child_consent.save()

    old_apps = None
    a_ons = None

    try:
        a_ons = OnScheduleCohortCEnrollment.objects.get(
            subject_identifier=child_consent.subject_identifier[:-3])
    except OnScheduleCohortCEnrollment.DoesNotExist:
        print('whoa!', child_consent.subject_identifier[:-3])
    else:
        old_apps = Appointment.objects.filter(subject_identifier=a_ons.subject_identifier,
                                             schedule_name=a_ons.schedule_name,
                                             visit_code_sequence=0)

    #check new enrollment onschedule model
    try:
        ons = OnScheduleCohortBEnrollment.objects.get(
            subject_identifier=child_consent.subject_identifier[:-3],
            child_subject_identifier=child_consent.subject_identifier)
    except:
        print('waittttt!', child_consent.subject_identifier[:-3])
    else:
        print('\n\nModify newly generated enrol appointments')
        
        print(old_apps)
        if old_apps:
            new_apps = Appointment.objects.filter(schedule_name=ons.schedule_name,
                                                  subject_identifier=ons.subject_identifier)
            for a in new_apps:

                ap = old_apps.get(visit_code=a.visit_code)


                try:
                    v1 = MaternalVisit.objects.get(appointment=ap)
                    v1.appointment = a
                    v1.save()
                except:
                    pass

                a.user_modified = ap.user_modified
                a.appt_status = ap.appt_status
                a.appt_reason = ap.appt_reason
                a.appt_datetime = ap.appt_datetime

                a.save()

            ssh = SubjectScheduleHistory.objects.get(subject_identifier=old_apps[0].subject_identifier,
                                                           schedule_name=old_apps[0].schedule_name)
            ssh.delete()

            initial_mv = MaternalVisit.objects.filter(subject_identifier=new_apps[0].subject_identifier,
                                                     schedule_name=new_apps[0].schedule_name).order_by(
                'report_datetime')[0]
            
            old_apps.delete()
            a_ons.delete()
    
            switch_to_cohortc_quart(initial_mv)



In [4]:
cccv = CaregiverChildConsent.objects.first()
cccv.child_age_at_enrollment

Decimal('7.10')

In [4]:
pids_2_move = CaregiverChildConsent.objects.filter(cohort='cohort_c', child_age_at_enrollment=10.5).values_list(
    'subject_identifier', flat=True).distinct()
#     child_dob__year='2012', child_dob__month__lt='2', cohort='cohort_b').values_list(
#     'subject_identifier', flat=True).distinct()

In [5]:
pids_2_move.count()

2

In [7]:
for pid in pids_2_move:
    switch_to_cohortc_enroll(pid, 'cohort_b')

Modify newly generated enrol appointments
<QuerySet [<Appointment: 2000M.0>]>
>>>>>>>>>>>>> c_quarterly1_schedule1
hmmmmm! B142-040990976-8
Modify newly generated enrol appointments
<QuerySet [<Appointment: 2000M.0>]>
>>>>>>>>>>>>> c_quarterly1_schedule1
hmmmmm! B142-040990639-2
Modify newly generated enrol appointments
<QuerySet [<Appointment: 2000M.0>]>
>>>>>>>>>>>>> c_quarterly1_schedule1
hmmmmm! B142-040990771-3
Modify newly generated enrol appointments
<QuerySet [<Appointment: 2000M.0>]>
>>>>>>>>>>>>> c_quarterly1_schedule1
hmmmmm! B142-040990790-3
Modify newly generated enrol appointments
<QuerySet [<Appointment: 2000M.0>]>
>>>>>>>>>>>>> c_quarterly1_schedule1
hmmmmm! B142-040990950-3
Modify newly generated enrol appointments
<QuerySet [<Appointment: 2000M.0>]>
>>>>>>>>>>>>> c_quarterly1_schedule1
hmmmmm! B142-040990766-3
Modify newly generated enrol appointments
<QuerySet [<Appointment: 2000M.0>]>
>>>>>>>>>>>>> c_quarterly1_schedule1
hmmmmm! B142-040990849-7
